In [ ]:
%pip install -q "transformers>=4.35.0" datasets soundfile torchaudio jiwer accelerate huggingface_hub wandb

In [ ]:
# ---------------- IMPORTS ----------------
import os, math, random, re
import numpy as np
import pandas as pd
import soundfile as sf
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, get_linear_schedule_with_warmup
from jiwer import wer
from tqdm.notebook import tqdm
import wandb
import torch
from torch.utils.data import Dataset

2025-09-28 11:21:54.138134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759058514.354052      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759058514.420625      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [9]:
import pandas as pd
from pathlib import Path

# Base recordings folder (contains train/, test/, val/)
DATASET_DIR = Path("/kaggle/input/medical-speech-transcription-and-intent/medical speech transcription and intent/Medical Speech, Transcription, and Intent/recordings")

# Load overview CSV
df = pd.read_csv(DATASET_DIR.parent / "overview-of-recordings.csv")

# Search all subfolders for each file
subfolders = ["train", "test", "validate"]
paths = []
missing = []

for fname in df['file_name']:
    found = False
    for sub in subfolders:
        full_path = DATASET_DIR / sub / fname
        if full_path.exists():
            paths.append(str(full_path))
            found = True
            break
    if not found:
        paths.append(None)
        missing.append(fname)

print(f"Missing files: {len(missing)}")
if missing:
    print(missing[:10])

# Build metadata for HuggingFace trainer
df_meta = pd.DataFrame({
    'path': paths,
    'transcript': df['phrase']
})

# Drop rows with missing files
df_meta = df_meta.dropna()

# Save metadata.csv in working dir
out_path = Path("/kaggle/working/metadata.csv")
df_meta.to_csv(out_path, index=False)
print("metadata.csv created at:", out_path)
print(df_meta.head())

Missing files: 0
metadata.csv created at: /kaggle/working/metadata.csv
                                                path  \
0  /kaggle/input/medical-speech-transcription-and...   
1  /kaggle/input/medical-speech-transcription-and...   
2  /kaggle/input/medical-speech-transcription-and...   
3  /kaggle/input/medical-speech-transcription-and...   
4  /kaggle/input/medical-speech-transcription-and...   

                                          transcript  
0                    When I remember her I feel down  
1  When I carry heavy things I feel like breaking...  
2          there is too much pain when i move my arm  
3  My son had his lip pierced and it is swollen a...  
4             My muscles in my lower back are aching  


In [10]:
import pandas as pd
import random
from IPython.display import Audio, display

# Load your metadata
metadata_path = "/kaggle/working/metadata.csv"
df_meta = pd.read_csv(metadata_path)

# Pick a random sample
sample = df_meta.sample(100).iloc[0]
audio_path = sample['path']
transcript = sample['transcript']

# Display audio player
print("Transcript:", transcript)
display(Audio(audio_path))


Transcript: I have shooting pain in my kneecap after working out.
